In [ ]:
# First make sure to install all required packages.
# You can do it by running the following command:

In [ ]:
# ]add Arrow CSV DataFrames Plots Clustering Distances FreqTables

In [ ]:
# If you launched Jupyter in directory with attached Project.toml and Manifest.toml
# use below command to install required packages with fixed versions. 
# Check Project introduction for more information.

In [ ]:
# ] instantiate

In [ ]:
# Import required libraries
using Arrow
using CSV
using DataFrames
using Plots
using Clustering
using Distances
using Random
using Statistics
using FreqTables

In [ ]:
# Read clean dataset from Arrow file into DataFrame
sales_norm = DataFrame(***("sales_norm.arrow"))

In [ ]:
# Convert DataFrame to Matrix and make sure that one product is one column in the resulting matrix
cluster_data = ***(sales_norm)

In [ ]:
# Calculate Euclidean distances between all points
dists = pairwise(SqEuclidean(), cluster_data)

In [ ]:
# Specify epsilons and minpts range
eps_rng = ***:***:***
minpts_rng = ***:***

In [ ]:
# Calculate average silhouettes values, number of clusters and mean clusters size 
#for each combination of hyperparameters
avg_shts_m = Array{Float64}[]
no_clust_m = Array{Int64}[]
avg_clust_size_m = Array{Float64}[]
for m in minpts_rng
    avg_shts = Float64[]
    no_clust = Int64[]
    avg_clust_size = Float64[]
    for e in eps_rng
        clustering = ***(dists, e, m)
        cluster_ids = ***
        if ***(cluster_ids) <= 1
            silh = ***
        else
            idx = cluster_ids .!=0
            silh = ***(***(cluster_ids[idx], dists[idx, idx]))
        end
        push!(avg_shts, silh)
        push!(no_clust, ***(***))
        push!(avg_clust_size, ***(***))
    end
    push!(avg_shts_m, avg_shts)
    push!(no_clust_m, no_clust)
    push!(avg_clust_size_m, avg_clust_size)
end

In [ ]:
# Plot average silhouettes values
plot(***, ***, xlab = "epsilon", ylab = "Mean silhouette value", 
    labels = permutedims(["m="*string(m) for m in 1:6]), linewidth = 2)

In [ ]:
# Plot number of clusters
plot(***, ***[***], xlab = "epsilon", ylab = "Number of clusters", 
    labels = permutedims(["m="*string(m) for m in 2:6]), linewidth = 2)

In [ ]:
# Plot average clusters size
plot(***, ***[***], xlab = "epsilon", ylab = "Average cluster size",
        labels = permutedims(["m="*string(m) for m in 2:6]), linewidth = 2, legend=:topleft)

DBSCAN is producing really small clusters up to 40.0 value of epsilon - we want to have few clusters we can analyze and write down the findings. Getting back to the average silhouette plot, we can se that for epsilon > 40.0 there is a spike around epsilon equal to 50.0 for m=5 or m=6. 

We'll use epsilon=50.0 and m=6 as hyperparameter values for further analysis.

In [ ]:
# Produce final DBSCAN clustering
opt_clustering = ***(dists, 50.0, 6)

In [ ]:
# Check the members count in each cluster
# There are 458 products considered as outliers
#observations are dispersed in high dimensional space and don't create dense areas
***(opt_clustering.assignments)

In [ ]:
# Plot clusters averages (also include outliers as separate cluster)
# Each cluster has some distinct characteristic we should summarize for the recipients of the report
plot(hcat([mean(cluster_data[:, *** .== ***], dims=2) for i in 0:2]...), 
    xlab="Week", ylab="Normalized sales", labels=[0 1 2], linewidth=2)

In [ ]:
# Plot clusters standard deviations
# Variability visibly differs between groups
plot(hcat([std(cluster_data[:, *** .== ***], dims=2) for i in 0:2]...), 
    xlab="Week", ylab="Normalized sales", labels=[0 1 2], linewidth=2)

In [ ]:
# Load k-means assignments from Milestone 2
kmeans_assignments = ***.(Int64,***("kmeans_assignments.txt"));

In [ ]:
# Produce two-way table for DBSCAN and k-means assignments
# As we can see, Cluster 2 from DBSCAN consists almost only of observations from k-means Cluster 4
# Outliers and Cluster 1 from DBSCAN have no clear mapping to k-means while considering observations membership 
#(but we still see similar shape on average sales plot between DBSCAN Cluster 1 and k-means Cluster 1) 
***(opt_clustering.assignments,kmeans_assignments)

**Analysis of clustering results**

Based on DBSCAN clustering evaluation we produced *** clusters with *** observations in total (we removed *** products as outliers).

* Cluster 1 ***
* Cluster 2 ***
* Outliers ***

Variance analysis shows that ***

We produced two distinct clusterings using `k-means` and `DBSCAN` methods. *** . Considering results from both clustering methods we can also see similar patterns for particular clusters:
* DBSCAN Cluster 1 is similar to ***
* DBSCAN Cluster 2 is alike ***

Such similiarties strengthen our confidence in quality and meaningfulness of the results. Both clustering results may be relevant for product team and person with in-depth knowledge about the products may provide additional comments regarding clustering quality. 

As a final technical detail, we want to highlight that our goal was to spot distinct groups in the product base regardless of the density of these groups. DBSCAN catch dense regions well, but related products are not guaranteed to be close to one another. We consider k-means to be our main analysis technique and DBSCAN as a validation tool.